#  VacationPy

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key

### Store Part I results into DataFrame

In [2]:
city_data = pd.read_csv("../WeatherPy/Resources/WeatherPy.csv")
city_data.head()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Makakilo City,90,US,1584178201,93,21.35,-158.09,73.40,5.82
1,Camacha,75,PT,1584178201,63,33.08,-16.33,60.80,14.99
2,Nikolskoye,20,RU,1584178201,46,59.70,30.79,28.40,20.13
3,Rikitea,30,PF,1584178201,71,-23.12,-134.97,77.77,9.86
4,Ribeira Grande,40,PT,1584178201,82,38.52,-28.70,59.00,2.24


### Humidity Heatmap

In [3]:
gmaps.configure(api_key = g_key)
locations = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"]
fig = gmaps.Map()
fig = gmaps.figure(center=(25.0, 5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

###  Create new DataFrame fitting weather criteria

In [4]:
optimal_vac = city_data.loc[city_data['Max Temp'] <= 85, :]
optimal_vac = optimal_vac.loc[optimal_vac['Max Temp'] >= 65, :]
optimal_vac = optimal_vac.loc[optimal_vac['Humidity'] <= 60, :]
optimal_vac = optimal_vac.loc[optimal_vac['Humidity'] >= 30, :]
optimal_vac = optimal_vac.loc[optimal_vac['Wind Speed'] <= 10, :]
optimal_vac = optimal_vac.loc[optimal_vac['Cloudiness'] <= 50, :]
optimal_vac.dropna()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
66,Jalu,32,LY,1584178212,43,29.03,21.55,67.51,9.26
78,Hithadhoo,4,MV,1584178213,55,-0.60,73.08,83.88,3.78
148,Korem,39,ET,1584178222,56,12.51,39.52,69.37,9.75
161,Hong Kong,20,HK,1584178133,53,22.29,114.16,77.00,8.05
195,Chiman,40,PK,1584178228,56,32.55,73.81,68.00,4.70
260,Haridwar,8,IN,1584178238,41,29.97,78.17,75.51,7.72
288,Masunga,1,BW,1584178241,44,-20.62,27.45,83.68,3.20
312,Djambala,0,CG,1584178245,55,-2.54,14.75,84.67,3.60
368,Upington,0,ZA,1584178203,39,-28.45,21.26,80.60,1.12


### Hotel Map

In [5]:
hotel_df = optimal_vac
hotel_df['Hotel Name'] = ''
hotel_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
66,Jalu,32,LY,1584178212,43,29.03,21.55,67.51,9.26,
78,Hithadhoo,4,MV,1584178213,55,-0.60,73.08,83.88,3.78,
148,Korem,39,ET,1584178222,56,12.51,39.52,69.37,9.75,
161,Hong Kong,20,HK,1584178133,53,22.29,114.16,77.00,8.05,
195,Chiman,40,PK,1584178228,56,32.55,73.81,68.00,4.70,
260,Haridwar,8,IN,1584178238,41,29.97,78.17,75.51,7.72,
288,Masunga,1,BW,1584178241,44,-20.62,27.45,83.68,3.20,
312,Djambala,0,CG,1584178245,55,-2.54,14.75,84.67,3.60,
368,Upington,0,ZA,1584178203,39,-28.45,21.26,80.60,1.12,


In [6]:
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

hotel_name = []

idx = 0
for index, row in hotel_df.iterrows():
    params = {
        "location": f"{row['Latitude']}, {row['Longitude']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

narrowed_city_df = hotel_df

In [7]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]
narrowed_city_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
66,Jalu,32,LY,1584178212,43,29.03,21.55,67.51,9.26,Jalu Hotel
78,Hithadhoo,4,MV,1584178213,55,-0.60,73.08,83.88,3.78,Pebbles Inn
148,Korem,39,ET,1584178222,56,12.51,39.52,69.37,9.75,
161,Hong Kong,20,HK,1584178133,53,22.29,114.16,77.00,8.05,Hotel ICON
195,Chiman,40,PK,1584178228,56,32.55,73.81,68.00,4.70,
260,Haridwar,8,IN,1584178238,41,29.97,78.17,75.51,7.72,Hotel Ashok
288,Masunga,1,BW,1584178241,44,-20.62,27.45,83.68,3.20,Spoponas Hotel
312,Djambala,0,CG,1584178245,55,-2.54,14.75,84.67,3.60,Hotel La Victoire
368,Upington,0,ZA,1584178203,39,-28.45,21.26,80.60,1.12,Protea Hotel by Marriott Upington


In [8]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))